# Classification model training script
Goal of model is to classify action between attack/explore

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.optim as optim
import torch.nn as nn
from sklearn.metrics import accuracy_score


from utility_modules.torch_toolkit import ClassificationCNN, ClassificationCNN_type2, ClassificationDataset

In [2]:
folder_paths = ['augmented_data/', 'classification_data/', 'predicted_data/']

dataset = ClassificationDataset(folder_paths)
print(f'created dataset, with {len(dataset)} images')
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)



created dataset, with 7206 images
Compose(
    ToTensor()
    Resize(size=(360, 640), interpolation=bilinear, max_size=None, antialias=True)
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
)


## CNN structure:
Model consists of 3 convolutional layers, 1 max pooling layer to lower computation cost, and 3 fully connected layers, with output layer of 2 neurons attack/explore. <br>

The input for the model is 640x360 tensor after convolutional layers results in 64 feature maps with size 45x80(0.125 * 640x480).

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')
model = ClassificationCNN().to(device)

criterion = nn.BCEWithLogitsLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)

Using device: cuda


In [4]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')


Epoch [1/5], Loss: 0.6453
Epoch [2/5], Loss: 0.6118
Epoch [3/5], Loss: 0.5946
Epoch [4/5], Loss: 0.5853
Epoch [5/5], Loss: 0.5728


In [9]:
# Evaluation loop
true_labels = []
predicted_labels = []
model.eval()
with torch.no_grad():
    val_loss = 0.0
    correct = 0
    total = 0
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()
        predicted = torch.round(torch.sigmoid(outputs))
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

accuracy = accuracy_score(true_labels, predicted_labels)

print(f'Accuracy: {accuracy:.4f}')


Accuracy: 0.6553


## Further training of already existent model

In [10]:
torch.save(model.state_dict(), 'class_torch_7kds.pth')

## Type 2 classification model
Lowered neurons in layer from 512 to 128, and added extra fully connected layer. Type 2 model has 4 fully connected layers with 128, 64, 32, 2 neurons respectfully.

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')
model = ClassificationCNN_type2().to(device)

criterion = nn.BCEWithLogitsLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)

Using device: cuda


In [4]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')


Epoch [1/5], Loss: 0.6343
Epoch [2/5], Loss: 0.6090
Epoch [3/5], Loss: 0.6035
Epoch [4/5], Loss: 0.5952
Epoch [5/5], Loss: 0.5859


In [5]:
true_labels = []
predicted_labels = []
model.eval()
with torch.no_grad():
    val_loss = 0.0
    correct = 0
    total = 0
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()
        predicted = torch.round(torch.sigmoid(outputs))
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

accuracy = accuracy_score(true_labels, predicted_labels)

print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.6692


## Type 2 shows good results!!
Each train epoch is 7 times faster, takes 2.2 minutes. Accuracy is slightly higher, so I will try to train the model few more epochs.

In [6]:
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

Epoch [1/15], Loss: 0.5820
Epoch [2/15], Loss: 0.5711
Epoch [3/15], Loss: 0.5582
Epoch [4/15], Loss: 0.5505
Epoch [5/15], Loss: 0.5417
Epoch [6/15], Loss: 0.5389
Epoch [7/15], Loss: 0.5331
Epoch [8/15], Loss: 0.5309
Epoch [9/15], Loss: 0.5313
Epoch [10/15], Loss: 0.5279
Epoch [11/15], Loss: 0.5268
Epoch [12/15], Loss: 0.5266
Epoch [13/15], Loss: 0.5288
Epoch [14/15], Loss: 0.5274
Epoch [15/15], Loss: 0.5279


In [7]:
true_labels = []
predicted_labels = []
model.eval()
with torch.no_grad():
    val_loss = 0.0
    correct = 0
    total = 0
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()
        predicted = torch.round(torch.sigmoid(outputs))
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

accuracy = accuracy_score(true_labels, predicted_labels)

print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.6796


In [8]:
torch.save(model.state_dict(), 'class_torch_type2.pth')